In [1]:
%matplotlib inline
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import cm

from scipy.stats import multivariate_normal

In [5]:
from pyMBstat.HIM import HIM

# The Harmonic Interaction Model

### The N-body probabilty distribution function (PDF)

\begin{align} 
P_m &\equiv P(r_1,r_2,\ldots,r_m) = \left|\Psi\left(r_1,r_2,\ldots,r_N\right)\right|^2 \\
&= \left(\frac{\delta_N}{\pi}\right)^{\frac{dm}{2}}\left(\frac{N\omega}{(N-m)\omega +m\delta_N}\right)^{\frac{d}{2}}\exp\left(-\frac{1}{N}\left(\omega+(N-1)\delta_N\right)\sum_{i=1}^mr_i^2-\frac{2}{N}
\sum_{i<j}^mr_ir_j-C_m\left(\sum_{i=1}^mr_i\right)^2\right)
\end{align}


#### where
\begin{equation*}
C_m = -\frac{1}{N}\frac{(N-m)(\omega-\delta_N)^2}{(N-m)\omega + m \delta_N} 
\end{equation*}

### Probability "Chain Rule"

\begin{equation*}
P(r_1,r_2,\ldots,r_m) = P(r_1)P(r_2|r_1)\cdots P(r_m|r_1,r_2,\ldots,r_{m-1})
\end{equation*}

### The conditional probability

\begin{align} 
P(r_m|r_1,r_2,\ldots,r_{m-1}) &= \frac{P(r_1,r_2,\ldots,r_m)}{P(r_1,r_2,\ldots,r_{m-1})}= \frac{P_m}{P_{m-1}}\\ &= \left(\frac{\delta_N}{\pi}\right)^{\frac{d}{2}}\left(1 + \frac{\omega-\delta_N}{(N-m)\omega +m\delta_N}\right)^{\frac{d}{2}}\exp\left(-\frac{1}{N}(\omega+(N-1)\delta_N)r_m^2 -\frac{1}{N}r_m\sum_{i=1}^{m-1}r_i\right)\\
&\times\exp\left( -C_mr_m^2-2C_mr_m\sum_{i=1}^{m-1}r_i-\left(\sum_{i=1}^{m-1}r_i\right)^2\left(C_m-C_{m-1}\right) \right) \\
&= \frac{1}{\sqrt{2\pi\sigma_m^2}}\exp\left(-\frac{\left(r_m-\mu_m\right)^2}{2\sigma_m^2} \right)
\end{align}


#### where
\begin{equation*}
\mu_m = \frac{\omega-\delta_N}{(m-1)\delta_N+(1-m+N)\omega}\sum_{i=1}^{m-1}r_i 
\end{equation*}
\begin{equation*}
\sigma_m^2 = \frac{m(\delta_N-\omega)+N\omega}{2\delta_N\left((m-1)\delta_N+(1-m+N)\omega\right)} 
\end{equation*}



In [6]:
system = HIM(N = 2, omega = 1, lambda0 = 5);
#print(system.__doc__)
#system.check_condP()

In [7]:
rho1hist = system.rho1hist(numshots = 100000)

Estimated Sampling Efficency=14.46%


In [ ]:
plt.hist(rho1hist,normed = 1, bins = 32, alpha = 0.5, histtype='stepfilled', linewidth = 3, color = 'b');
plt.plot(x,  system.rho1(x), linewidth = 3, color = 'k');

In [ ]:
sshot = system.singleshot(numshots = 100000)

In [ ]:
x = np.linspace(-3,3,512,endpoint=True)
plt.subplots(figsize = (12,8))
#plt.hist(sshot.ravel(),normed = 1, bins = 32, alpha = 0.5, histtype='stepfilled', linewidth = 3, color = 'b');
plt.hist(sshot[:,0],normed = 1, bins = 32, alpha = 0.5, histtype='stepfilled', linewidth = 3, color = 'b');
plt.plot(x,  system.rho1(x), linewidth = 3, color = 'k');
plt.plot(x,  system.conditonalP(x,0,2),'--', linewidth = 3, color = 'r');
#plt.plot(x,  system.conditonalP(x,0,3),':', linewidth = 3, color = 'g');
#plt.plot(x,  system.conditonalP(x,0,4),'--', linewidth = 3, color = 'm');

In [ ]:
system = HIM(N = 2, omega = 1, lambda0 = 1)
#sshot = system.singleshot(numshots = 10000)
rho2hist = system.rho2hist(numshots = 100000)

In [ ]:
#H, edges = np.histogramdd(sshot, bins = (32,32))
H, edges = np.histogramdd(rho2hist, bins = (32,32))
fig, axs = plt.subplots(1,2,figsize = (12,6))
H = H / len(rho2hist[:,0])
axs[0].pcolor(H)        
xvec = np.linspace(-4,4,32,endpoint=True)
Pxy = np.zeros((len(xvec),len(xvec)))
for i,x in enumerate(xvec):
    for j,xp in enumerate(xvec):
        Pxy[i,j] = system.rho2(x,xp)
#[X1,X2] = np.meshgrid(x,x)
axs[1].pcolor(Pxy);


In [ ]:
rho2hist.shape

In [ ]:
type(rho2hist)

In [ ]:
sum(sum(H))

In [ ]:
sum(sum(Pxy))*(xvec[1]-xvec[0])**2

In [ ]:
H, xedges, yedges = np.histogram2d(, y, bins=(xedges, yedges))
H = H.T  # Let each row list bins with common y range.
plt.pcolor(H/)

In [ ]:
sshot.shape

In [ ]:
hist, bins = np.histogram(sshot[:,0], density=False, bins = 32)

width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
hist_size = sum(hist)
plt.bar(center, hist/hist_size, align='center', width=width);

hist, bins = np.histogram(sshot[:,1], density=False, bins = 32)

width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
hist_size = sum(hist)
plt.bar(center, hist/hist_size, align='center', width=width);

#sshot_2_rho = sshot[:,0]*sum(sshot[:,1])/sshot.shape[0]

In [ ]:
sum(hist/np.sqrt(hist_size))*0.14821015

In [ ]:
np.diff(bins)

In [ ]:
plt.subplots(figsize = (12,8))
#plt.hist(sshot[:,0],normed = 1, bins = 32, alpha = 0.5, histtype='stepfilled', linewidth = 3, color = 'b');
#plt.plot(x,  system.rho1(x), linewidth = 3, color = 'k');
#plt.plot(x,  system.conditonalP(x,0,2),'--', linewidth = 3, color = 'r');

In [ ]:
#x = np.random.random_sample(1000)
#y = np.random.random_sample(1000)
x = np.random.randn(10000)
y = np.random.randn(10000)
H, xedges, yedges = np.histogram2d(x, y, bins=(32, 32))
H = H.T  # Let each row list bins with common y range.

fig = plt.figure(figsize=(12, 8))
plt.pcolor(H);

In [ ]:
[X,Y] = np.meshgrid(x,y)
P = 1/(1 * np.sqrt(2 * np.pi)) * np.exp( - (X - 0)**2 / (2 * 1**2)) *1/(1 * np.sqrt(2 * np.pi)) * np.exp( - (Y - 0)**2 / (2 * 1**2)) 
plt.pcolor(P);